# List of Exposures with Hologram at USDF


- work with Weakly_2022_39
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/09/19
- update : 2022/10/31



In [1]:
import sys
print(sys.executable)

/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-4.1.0/bin/python3


In [2]:
print(sys.path)

['/home/d/dagoret/notebooks/AuxTelComm/notebooks_usdf/butlertools', '/home/d/dagoret/repos/repos_w_2022_39/summit_utils/python', '/home/d/dagoret/repos/repos_w_2022_39/drp_pipe/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/analysis_tools/gb9b489fd42+90dca76f06/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/analysis_drp/g12d5fbe6a5+90dca76f06/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/faro/gbb36616a4b+5e2d81108f/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/fgcmcal/g5147cf65fe+deeae05922/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/fgcm/g3dbc634184/lib/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/jointcal/ge7491f621d+ceb865f59e/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/obs_cfht/gd4e7b81488+e2797d1921/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1

In [3]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2022_39/atmospec 	setup
drp_pipe              LOCAL:/home/d/dagoret/repos/repos_w_2022_39/drp_pipe 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-4.1.0/eups 	setup
summit_utils          LOCAL:/home/d/dagoret/repos/repos_w_2022_39/summit_utils 	setup


In [4]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib

w_2022_39
   g0b29ad24fb+a10408d0bf 	current w_2022_39 setup


In [5]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits

In [6]:
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [7]:
import lsst.daf.butler as dafButler

In [9]:
#repo = "/sdf/group/rubin/repo/main"
repo="/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [ ]:
collection='LATISS/raw/all'

In [ ]:
# loop on exposures
for idx in range(N):
    exposure_selected=df_bias_this_date.iloc[idx].id 
    
   
    raw_img = butler.get('raw', dataId={'exposure': exposure_selected, 'instrument': 'LATISS', 'detector': 0}, collections = collection)

    fig = plt.figure(figsize=(12,10))
    afw_display = afwDisplay.Display(frame=fig)
    afw_display.scale('linear', 'zscale',None)
    title = f"raw flat : {exposure_selected}"
    afw_display.mtv(raw_img.image,title=title)
    #plt.gca().axis('off')

## List of Exposures

In [10]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs', 'seq_num', 'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle'])

In [11]:
for count, info in enumerate(registry.queryDimensionRecords('exposure',where= "instrument='LATISS'")):
    
    
    df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num, info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle]
    
    if count < 0:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
    

In [12]:
df_exposure

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
0,2022083100004,AT_O_20220831_000004,20220831,4,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN
1,2022083100008,AT_O_20220831_000008,20220831,8,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN
2,2022083100007,AT_O_20220831_000007,20220831,7,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN
3,2022083100009,AT_O_20220831_000009,20220831,9,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN
4,2022083100012,AT_O_20220831_000012,20220831,12,dark,UNKNOWN,unknown~unknown,NaN,3.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
27347,2022120700461,AT_O_20221207_000461,20221207,461,acq,HD074000,empty~holo4_003,38.894793,5.0,130.201416,-16.312413,331.84383
27348,2022120700463,AT_O_20221207_000463,20221207,463,science,HD074000,empty~holo4_003,38.771097,30.0,130.201339,-16.312296,331.843653
27349,2022120700465,AT_O_20221207_000465,20221207,465,cwfs,HD 39240,SDSSr_65mm~empty,6.866105,30.0,87.557033,-24.111028,72.262111
27350,2022120700467,AT_O_20221207_000467,20221207,467,cwfs,HD 39240,SDSSr_65mm~empty,7.052928,30.0,87.556951,-24.111017,72.261998


## Selection of science

In [13]:
df_science = df_exposure[df_exposure.type == 'science']

In [14]:
df_science.sort_values(by="id",ascending=True,inplace=True)

/tmp/ipykernel_3929/3986782663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science.sort_values(by="id",ascending=True,inplace=True)


In [15]:
df_science.tail(60)

,id,obs_id,day_obs,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle
27229,2022120700332,AT_O_20221207_000332,20221207,332,science,SPT-CL-J0559-5249_040,SDSSr_65mm~empty,28.835273,30.0,90.392546,-52.825922,359.998185
27202,2022120700333,AT_O_20221207_000333,20221207,333,science,SPT-CL-J0559-5249_040,SDSSr_65mm~empty,28.774951,30.0,90.392426,-52.825905,359.998017
27230,2022120700334,AT_O_20221207_000334,20221207,334,science,SPT-CL-J0559-5249_040,SDSSg_65mm~empty,28.692466,30.0,90.392478,-52.82595,359.998188
27203,2022120700335,AT_O_20221207_000335,20221207,335,science,SPT-CL-J0559-5249_040,SDSSg_65mm~empty,28.630264,30.0,90.391868,-52.825554,359.998059
27231,2022120700336,AT_O_20221207_000336,20221207,336,science,SPT-CL-J0559-5249_040,SDSSi_65mm~empty,28.550669,30.0,90.392343,-52.825895,359.998232
27204,2022120700337,AT_O_20221207_000337,20221207,337,science,SPT-CL-J0559-5249_040,SDSSi_65mm~empty,28.489579,30.0,90.392482,-52.825939,359.998176
27232,2022120700338,AT_O_20221207_000338,20221207,338,science,SPT-CL-J0559-5249_047,SDSSi_65mm~empty,28.271286,30.0,90.235964,-52.731864,359.99833
27205,2022120700339,AT_O_20221207_000339,20221207,339,science,SPT-CL-J0559-5249_047,SDSSi_65mm~empty,28.210546,30.0,90.235912,-52.731821,359.998266
27233,2022120700340,AT_O_20221207_000340,20221207,340,science,SPT-CL-J0559-5249_047,SDSSg_65mm~empty,28.131825,30.0,90.235949,-52.731845,359.998243
27206,2022120700341,AT_O_20221207_000341,20221207,341,science,SPT-CL-J0559-5249_047,SDSSg_65mm~empty,28.071514,30.0,90.235787,-52.731766,359.998292


In [16]:
df_science.day_obs.unique()

array([20220912, 20220913, 20220914, 20220927, 20220928, 20220929,
       20221011, 20221012, 20221013, 20221025, 20221026, 20221027,
       20221108, 20221109, 20221110, 20221122, 20221123, 20221124,
       20221207])

## Selection of date

2021-02-16  2021-02-18  2021-06-08  2021-06-10  2021-07-07  2021-09-07  2021-09-09  2021-10-06  2021-11-02  2021-11-04  2022-02-16  2022-03-15  2022-03-17  20220502  
2021-02-17  2021-03-11  2021-06-09  2021-07-06  2021-07-08  2021-09-08  2021-10-05  2021-10-07  2021-11-03  2022-02-15  2022-02-17  2022-03-16  2022-03-18  20220503 20220524 20220607 20220608 20220609

### Add those

20220504 - Not existing

In [17]:
#date_sel = 20220630
date_sel = 20221207

In [18]:
df_science_selected = df_science[df_science.day_obs == date_sel ]

In [19]:
len(df_science_selected)

360

# List of filters

In [20]:
list_of_filters = df_science_selected['filter'].unique()
list_of_filters

array(['SDSSr_65mm~empty', 'SDSSg_65mm~empty', 'SDSSi_65mm~empty',
       'empty~holo4_003'], dtype=object)

In [21]:
selected_filters = []
for filt in list_of_filters:
    #flag_sel = (filt.find('holo4') != -1) or (filt.find('ronchi90lpmm') != -1) or (filt.find('ronchi170lpmm') != -1)
    flag_sel = (filt.find('holo4') != -1) 
    #flag_sel = (filt.find('ronchi170') != -1) 
    if flag_sel:
        selected_filters.append(filt) 
selected_filters = np.array(selected_filters)
selected_filters       

array(['empty~holo4_003'], dtype='<U15')

## Selection of filter

- But now we have a loop on filters

In [21]:
#idx_sel_filt = 0
#selected_filter = selected_filters[0]
#selected_filter

## Final selection of exposure

- loop on selected filters

In [22]:
pd.set_option('display.max_rows', None)

In [23]:
all_def_science_selected_final = []

for selected_filter in selected_filters:

    print(selected_filter)
    df_science_selected_final = df_science_selected[df_science_selected["filter"] ==  selected_filter]
    
    #print(df_science_selected_final)
    all_def_science_selected_final.append(df_science_selected_final.copy())
    
    df = df_science_selected_final
    df_sel = df[['day_obs', 'seq_num']]
    
    
    NN = len(df)
    
    if NN>0:
        # build the line
        
        df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
    
    
         # prepare output
        filename_out = "visitdispersers_"+ str(date_sel) + "_filt_" + selected_filter.replace('~','-') + ".list"
        print(filename_out)
        
        df_out = df_sel[["line"]]
        df_out.to_csv(filename_out, index=False, header=None)

empty~holo4_003
visitdispersers_20221207_filt_empty-holo4_003.list


/tmp/ipykernel_3929/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)


In [24]:
len(all_def_science_selected_final)

1

In [27]:
len(all_def_science_selected_final[0])

44

In [26]:
all_def_science_selected_final[1]

IndexError: list index out of range

In [ ]:
all_def_science_selected_final[2]

In [ ]:
all_def_science_selected_final[3]

In [ ]:
all_def_science_selected_final[4]